In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
import os
import time
import torch
from datasets import load_dataset
from transformers import (  # transformer
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    get_peft_model,
    PromptTuningConfig,
    LoraConfig,
    PeftModel,
    TaskType,
)

In [ ]:

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

model_path = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True,
    use_fast=False
)
# base_path = "/content/drive/MyDrive/model/dapt1"
# model = AutoModelForCausalLM.from_pretrained(base_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.float16,
).to('cpu')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config).cuda()
model.print_trainable_parameters()

trainable params: 1,081,344 || all params: 495,114,112 || trainable%: 0.2184


In [ ]:

def load_and_preprocess(data_path):
    dataset = load_dataset("json", data_files=data_path)

    dataset = dataset.filter(lambda x: len(x["prompt"]) > 0 and len(x["complete"]) > 0)

    def tokenize_function(examples):
        # print(examples)
        completions = [str(c) for c in examples["complete"]]

        texts = [p + c + tokenizer.eos_token for p, c in zip(examples["prompt"], completions)]

        tokenized = tokenizer(
            texts,
            max_length=512,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        prompt_tokenized = tokenizer(examples["prompt"], add_special_tokens=False)
        prompt_lengths = [len(ids) for ids in prompt_tokenized["input_ids"]]

        labels = tokenized["input_ids"].clone()
        for i, length in enumerate(prompt_lengths):
            if length >= 2048:
                length = 2047
            labels[i][:length] = -100

        tokenized["labels"] = labels
        return tokenized

    return dataset.map(tokenize_function, batched=True)

In [ ]:
def formal_train(json_path="salary_47_allin.json", target_model_path="./qwen_prompt_47"):
    dataset = load_and_preprocess(json_path)
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir="./qwen_prompt_tuning_output",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        num_train_epochs=6,
        learning_rate=8e-4,
        logging_dir="./logs",
        logging_steps=100,
        save_strategy="epoch",
        fp16=True,
        optim="adamw_torch",
        dataloader_num_workers=4,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        data_collator=data_collator,
    )

    print("开始训练...")
    start_time = time.time()
    trainer.train()
    print(f"训练完成，耗时: {time.time() - start_time:.2f}秒")

    model.save_pretrained(target_model_path)


In [ ]:
formal_train(json_path="./drive/MyDrive/datasetfiles/agument_work_5316.json", target_model_path="./drive/MyDrive/model/nodapt_agument_work_5316")

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/5316 [00:00<?, ? examples/s]

Map:   0%|          | 0/5316 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


开始训练...


Step,Training Loss
100,2.842300
200,2.718700
300,2.664900
400,2.613600
500,2.595500
600,2.589300
700,2.562200
800,2.524000
900,2.508900
1000,2.526300


训练完成，耗时: 613.96秒


In [ ]:

def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=100,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
    return tokenizer.decode(generated_tokens, skip_special_tokens=True)

In [ ]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_features

In [ ]:
test_path = "./drive/MyDrive/datasetfiles/test_1355.json"
# Load val_data from JSON file
import json
with open(test_path, 'r') as f: #Update with the path to your JSON File.
    data = json.load(f)

import time

t0 = time.time()
answers = []
for i, item in enumerate(data):
  # if (0 <= i < 20) or (567 <= i < 587) or (1256 <= i):
    # print("=" * 20, 'Round:', i, "=" * 20)
    p = item['prompt']
    # y = item['complete']
    # print("Target:", y)
    # print("\n Prompt tuning Answer:", generate_response(p))
    answers.append(generate_response(p))
t1 = time.time()
print("Inference time:", t1 - t0)

import pandas as pd

df = pd.DataFrame(data)
df["y_pred"] = answers

output_path = "./drive/MyDrive/datasetfiles/LoRA_05B_work_aug_results.json"
df.to_json(output_path, orient='records', indent=4, force_ascii=False)

Inference time: 384.2383120059967
